In [1]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 01. Criando uma Vector Database Local (em disco)

In [2]:
path_ITR = "03. Data/itr.pdf"
path_chroma_database = "02. Databases/stage_chroma_db"

In [3]:
loader = PyPDFLoader(path_ITR)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [16]:
label_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
<Instrução>
Classifique o chunk a seguir como uma das seguintes categorias:
- Empresa e estrutura
- Operações e mercado
- Financeiro e contábil
- Riscos e conformidade
- ESG e sustentabilidade
- Recursos humanos e administração
- Relação com investidores e mercado de capitais
<Instrução>


<Exemplos>
Chunk:  A Petrobras foi constituída pela União Federal, através da Lei nº 2.004, de 3 de outubro de 1953, sob forma de 
sociedade por ações, a fim de conduzir as atividades correlatas à indústria de petróleo e gás natural pelo governo 
brasileiro. A Companhia começou suas operações em 1954 e durante aproximadamente quarenta anos 
Output: Empresa e estrutura

Chunk: A Petrobras também aumentará a capacidade de produção de Diesel S10 em 290 mil bpd em seu parque de refino, 
considerando os projetos da Carteira de Implantação. Além disso, contará com sua primeira unidade de lubrificantes Grupo II 
(mais modernos), com capacidade de 12 mil bpd até 2029. Com os projetos da Carteira de
Output: Operações e mercado

Chunk: As atividades relacionadas ao negócio de petróleo e gás apresentam riscos elevados, geralmente por envolverem 
altas temperaturas e pressões. Em particular, atividades em águas profundas e ultraprofundas, no refino e em 
petroquímica, realizadas pela Companhia, suas subsidiárias, ou empresas coligadas apresentam vários riscos, como 
vazamentos de óleo e produtos, colapsos, acidentes aeronáuticos, incêndios e ex
Output: Riscos e conformidade
<Exemplos>


<Chunk>
{text}
<Chunk>

Responda somente a categoria.
Category:"""
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [17]:
for i, chunk in enumerate(chunks):
    if i>10: break
    print(f"{i} / {len(chunks)}", end=" ...")
    category = llm.invoke(label_prompt.invoke({"text":chunk.page_content}))
    chunk.metadata["category"] = category.content.strip()
    print(category.content.strip())

0 / 2533 ...Relação com investidores e mercado de capitais
1 / 2533 ...Operações e mercado
2 / 2533 ...Financeiro e contábil
3 / 2533 ...Riscos e conformidade
4 / 2533 ...Recursos humanos e administração
5 / 2533 ...Recursos humanos e administração
6 / 2533 ...Recursos humanos e administração
7 / 2533 ...Relação com investidores e mercado de capitais
8 / 2533 ...Relação com investidores e mercado de capitais
9 / 2533 ...Empresa e estrutura
10 / 2533 ...Operações e mercado


In [ ]:
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=path_chroma_database,
    collection_name="MyColletion"
)
